In [1]:
import xarray as xr
import glob
import netCDF4 as nc4


# based on this page:
http://www.smhi.net/hype/wiki/doku.php?id=start:hype_file_reference:nnobs.nc

In [2]:
# Set the folder path where your remapped .nc files are located and direct to MESH file
mesh_nc_file = '/home/shg096/scratch/Bow_at_Calgary/mesh_model/MESH_forcing.nc'

#
ds = xr.open_dataset(mesh_nc_file)

# rename the subbasin to id
ds = ds.rename({'subbasin': 'id'})
ds = ds.set_coords(['lat', 'lon'])

# drop variables
ds = ds.drop_vars(['RDRS_v2.1_P_UVC_10m',\
                   'RDRS_v2.1_P_FI_SFC',\
                   'RDRS_v2.1_P_FB_SFC',\
                   'RDRS_v2.1_P_HU_1.5m',\
                   'RDRS_v2.1_P_P0_SFC',\
                   'crs']) # ,'lat','lon'

# change time from UTC to approximate local time
time_diff = -7 # Alberta, BC

#
ds['time'] = ds['time'].roll(time=time_diff)

# Remove the first roll_steps time steps
if time_diff < 0:
    ds = ds.isel(time=slice( None, time_diff))
elif time_diff > 0:
    ds = ds.isel(time=slice( time_diff, None))

# sample to daily
ds_daily = ds.resample(time='D').mean()


# sample to daily to max, min, mean and sum of variables
ds_daily['TMAXobs'] = ds['RDRS_v2.1_P_TT_1.5m'].resample(time='D').max() - 273.15 # kelvin to C
ds_daily['TMINobs'] = ds['RDRS_v2.1_P_TT_1.5m'].resample(time='D').min() - 273.15 # kelvin to C
ds_daily['Tobs'] = ds['RDRS_v2.1_P_TT_1.5m'].resample(time='D').mean() - 273.15 # kelvin to C
ds_daily['Pobs'] = ds['RDRS_v2.1_P_PR0_SFC'].resample(time='D').sum()

ds_daily = ds_daily.drop_vars(['RDRS_v2.1_P_PR0_SFC',\
                               'RDRS_v2.1_P_TT_1.5m'])

# # Tmax
# TMAXobs = ds_daily
# TMAXobs = TMAXobs.drop_vars(['TMINobs','Tobs','Pobs'])
# TMAXobs.to_netcdf('/home/shg096/scratch/Bow_at_Calgary/HYPE/TMAXobs.nc')

# #Tmin
# TMINobs = ds_daily
# TMINobs = TMINobs.drop_vars(['TMAXobs','Tobs','Pobs'])
# TMINobs.to_netcdf('/home/shg096/scratch/Bow_at_Calgary/HYPE/TMINobs.nc')

# #T
# Tobs = ds_daily
# Tobs = Tobs.drop_vars(['TMAXobs','TMINobs','Pobs'])
# Tobs.to_netcdf('/home/shg096/scratch/Bow_at_Calgary/HYPE/Tobs.nc')

# #P
# Pobs = ds_daily
# Pobs = Pobs.drop_vars(['TMAXobs','TMINobs','Tobs'])
# Pobs.to_netcdf('/home/shg096/scratch/Bow_at_Calgary/HYPE/Pobs.nc')


In [3]:
ds_daily = ds_daily.drop_vars(['lat','lon'])

In [5]:
ds_daily_df_Tmax = ds_daily['TMAXobs'].to_dataframe()
ds_daily_df_Tmax = ds_daily_df_Tmax.unstack()
ds_daily_df_Tmax = ds_daily_df_Tmax.T
ds_daily_df_Tmax = ds_daily_df_Tmax.droplevel(level=0, axis=0)
ds_daily_df_Tmax.columns.name = None
ds_daily_df_Tmax.index.name = 'Date'
ds_daily_df_Tmax
ds_daily_df_Tmax.to_csv('/home/shg096/scratch/Bow_at_Calgary/HYPE/TMAXobs.txt',\
                        sep='\t', na_rep='', index_label='time', float_format='%.1f')


In [6]:
ds_daily_df_Tmin = ds_daily['TMINobs'].to_dataframe()
ds_daily_df_Tmin = ds_daily_df_Tmin.unstack()
ds_daily_df_Tmin = ds_daily_df_Tmin.T
ds_daily_df_Tmin = ds_daily_df_Tmin.droplevel(level=0, axis=0)
ds_daily_df_Tmin.columns.name = None
ds_daily_df_Tmin.index.name = 'Date'
ds_daily_df_Tmin
ds_daily_df_Tmin.to_csv('/home/shg096/scratch/Bow_at_Calgary/HYPE/TMINobs.txt',\
                        sep='\t', na_rep='', index_label='time', float_format='%.1f')


In [7]:
ds_daily_df_Tmean = ds_daily['Tobs'].to_dataframe()
ds_daily_df_Tmean = ds_daily_df_Tmean.unstack()
ds_daily_df_Tmean = ds_daily_df_Tmean.T
ds_daily_df_Tmean = ds_daily_df_Tmean.droplevel(level=0, axis=0)
ds_daily_df_Tmean.columns.name = None
ds_daily_df_Tmean.index.name = 'Date'
ds_daily_df_Tmean
ds_daily_df_Tmean.to_csv('/home/shg096/scratch/Bow_at_Calgary/HYPE/Tobs.txt',\
                         sep='\t', na_rep='', index_label='time', float_format='%.1f')


In [8]:
ds_daily_df_Pobs = ds_daily['Pobs'].to_dataframe()
ds_daily_df_Pobs = ds_daily_df_Pobs.unstack()
ds_daily_df_Pobs = ds_daily_df_Pobs.T
ds_daily_df_Pobs = ds_daily_df_Pobs.droplevel(level=0, axis=0)
ds_daily_df_Pobs.columns.name = None
ds_daily_df_Pobs.index.name = 'Date'
ds_daily_df_Pobs
ds_daily_df_Pobs.to_csv('/home/shg096/scratch/Bow_at_Calgary/HYPE/Pobs.txt',\
                         sep='\t', na_rep='', index_label='time', float_format='%.1f')